###### First Step is to download the CIFAR10 dataset (binary version used by Raptor-NN) to be used for quantizing and exporting the model

In [ ]:
%%sh
../Shared/download_cifar10_dataset.sh

## 1. First Workflow
### 1.1. Training the model with Keras

We start by importing packages that we're going to use

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf

from keras_model_def import resnet_v1
from data_loader import load_cifar_10_data

We then define global parameters that can be used when training

In [ ]:
data_path = "../Shared/cifar-10-batches-py/"
num_classes = 10
num_filters = 16
image_size = 32
batch_size = 16
num_epochs = 10
resume_train = False
optimizer_name = "adam"
model_save_dir = "img_cls_trained_model_adam_10epochs"

In [ ]:
# Loading either an already trained model or instantiate a new one
if resume_train:
    keras_model = tf.keras.models.load_model(os.path.join('trained_models', model_save_dir)) 
else: 
    keras_model = resnet_v1(input_shape=[image_size, image_size, 3], 
                num_classes=num_classes, num_filters=num_filters)

In [ ]:
# Printing the model summary
keras_model.summary()

In [ ]:
# Loading training and test data using a custom loader
train_data, train_labels, test_data, test_labels = load_cifar_10_data(data_path)

In [ ]:
# Applying data augmentation (transformations) using Keras's ImageDataGenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                rotation_range=10,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                horizontal_flip=True,
                                rescale=1./255
                                )

In [ ]:
# Instantiating the train and validation generators
train_generator = datagen.flow(train_data, train_labels, batch_size=batch_size)
valid_generator = datagen.flow(test_data, test_labels, batch_size=batch_size)

In [ ]:
# Defining an optimizer to be used for training (commonly used ones are SGD and Adam)
if optimizer_name == "sgd":
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, name='SGD')
else:
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, name='Adam')

In [ ]:
# Defining the loss function and metrics for this network
loss_func = 'categorical_crossentropy'
metrics = ['accuracy']

In [ ]:
# Compiling the model for training
keras_model.compile(optimizer=optimizer, loss=loss_func, metrics=metrics)

In [ ]:
# Starting the training of the model for a certain number of epochs
history = keras_model.fit(
            train_generator,
            epochs=num_epochs,
            steps_per_epoch=len(train_generator),
            validation_data=valid_generator,
            validation_steps=len(valid_generator),
            shuffle=True,
            verbose=2)

In [ ]:
# Plotting and saving accuracy/loss curves
plt.plot(np.array(range(num_epochs)), history.history['loss'])
plt.plot(np.array(range(num_epochs)), history.history['accuracy'])
plt.savefig('tf_loss_accuracy_curves.png')

In [ ]:
# Saving the model to a directory
keras_model.save(os.path.join('trained_models', model_save_dir))

### 1.2. Exporting the model with Raptor-NN

We start by importing packages that we're going to use

In [ ]:
import tensorflow as tf
import keras_to_n2d2
import n2d2
import n2d2_ip
import os

We then define global parameters that can be used when exporting

In [ ]:
data_path = "../Shared/cifar-10-batches-bin/"
image_size = 32
batch_size = 32
saved_model_dir = "img_cls_trained_model_adam_2epochs"

In [ ]:
# (Optional) We can load an already trained Keras model to be wrapped by Raptor-NN
keras_model = tf.keras.models.load_model(os.path.join('trained_models', saved_model_dir))

In [ ]:
# Wrapping the Keras model with N2D2 Backend
raptor_model = keras_to_n2d2.wrap(keras_model, batch_size=batch_size,
                            name="resnet_model", for_export=True)

In [ ]:
# Loading CIFAR10 database that will be used for calibration / quantization (test = 1 - learn - validation).
db = n2d2.database.CIFAR10()
db.load(data_path=data_path)

In [ ]:
# Printing the train, validation and test partition summary
db.get_partition_summary()

In [ ]:
# Initiating the data provider for data loading
provider = n2d2.provider.DataProvider(db, [image_size, image_size, 3],
                                    batch_size=batch_size)

In [ ]:
# Creating a list of transformations that will be applied to images when loading

# Converting the images color space from BGR to RGB (Raptor-NN reads images in BGR format by default)
provider.add_transformation(n2d2.transform.ColorSpace(color_space="RGB"))

# Applying a transformation (Rescaling) as preprocessing
provider.add_transformation(n2d2.transform.RangeAffine(first_operator="Divides", first_value=255.0))

In [ ]:
# Returning the DeepnetCell that can be exported 
raptor_deepnet_cell = raptor_model.get_deepnet_cell()

In [ ]:
# (Optional) Removing Softmax layer (not supported) for CPP export
raptor_deepnet_cell.remove(raptor_deepnet_cell[-1].get_name())

In [ ]:
# Quantizing the network using PTQ (Post Training Quantization) quantizer
n2d2.quantizer.PTQ(
            raptor_deepnet_cell,
            provider=provider,
            nb_bits=8,
            nb_sitmuli=2000,
            no_unsigned=True,
            act_scaling_mode="SINGLE_SHIFT",
            export_no_cross_layer_equalization=True,
            )

In [ ]:
# A parameter for choosing the TinyRaptor export version
export_version = "v1"

In [ ]:
# Selecting the HW configuration file depending on the export version
if export_version == "v2":
    npu_params_file = "../Shared/npu_params_v2.ini"
else:
    npu_params_file = "../Shared/npu_params_v1.ini"
    
# Generating TinyRaptor Export
n2d2_ip.export.export_PNasm(
    raptor_deepnet_cell,
    npu_params_path=npu_params_file,
    export_nb_stimuli_max=1000
    )

In [ ]:
# Generating reference export (CPP)
n2d2.export.export_cpp(
    raptor_deepnet_cell,
    export_nb_stimuli_max=1000
    )

## 2. Second Workflow
### 2.1. Training the model with RaptorNN

### 2.2. Exporting the model with RaptorNN